# Exercise 5: Replica exchange umbrella sampling

In this exercise, our goal is to run replica exchange umbrella sampling (REUS) for the NaCl system to recover a free energy surface consistent with what we got from Exercise 2 (umbrella sampling) and Exercise 3 (multiple walkers metadynamics). Again, we will perform the simulation in the NVT ensemble. 

In [ ]:
%%bash
# Delete folders rep_*, if there is any. This is not necessary but convenient for rerunning the notebook. 
if ls -d rep_* >/dev/null 2>&1; then
  rm -rf rep_*
fi

## 1. Preparation of the input files

Here we will stick with 8 intermediate states (hence 8 replicas), with each of them having a different center for the umbrella potential. This requires us to set up 8 folders. 

In [ ]:
%%bash
for i in {0..7}
do
    mkdir rep_${i} && cd rep_${i}
    cp ../../Exercise_2/sim_${i}/NaCl_${i}.gro NaCl.gro   # configurations from the pulling ismulation in Exercise 2
    cp ../../Exercise_2/pull/NaCl_US.top NaCl.top
    cp ../../Inputs/NaCl/MD-NVT.mdp .
    cd ../
done

While it is possible to use only GROMACS to perform replica exchange umbrella sampling, it is easier to use the combination of GROMACS and PLUMED because PLUMED has much better flexibility in formulating the restraint. This means that we will need a different PLUMED input files for different replicas/folders.

In [ ]:
%%bash
module load gromacs/2020.2-cpu openmpi/4.0.5-gcc10.2.0
d=(0.25 0.3 0.35 0.4 0.45 0.5 0.55 0.6)  # centers
for i in {0..7}
do
    center=${d[$i]}
    echo "d: DISTANCE ATOMS=322,323
r: RESTRAINT ARG=d KAPPA=8000 AT=${center}
PRINT ARG=d,r.bias FILE=../colvar_multi.dat STRIDE=100
    " > plumed.dat
    mv plumed.dat rep_${i}/.
    cd rep_${i} && mpirun -np 1 gmx_mpi grompp -f MD-NVT.mdp -c NaCl.gro -p NaCl.top -o NaCl_REUS.tpr && cd ../
done

## 2. Running REUS simulations

In [ ]:
%%time
%%bash
module load gromacs/2020.2-cpu openmpi/4.0.5-gcc10.2.0

mpirun -np 8 gmx_mpi mdrun -s NaCl_REUS.tpr -plumed plumed.dat -multidir rep_{0..7} -replex 100 -ntomp 1

## 3. Data analysis

In [ ]:
%%bash
ls *dat

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
pullx_data = [np.transpose(np.loadtxt(f'colvar_multi.{i}.dat', comments=['@', '#'])) for i in range(8)]
dist_list = [data[1] for data in pullx_data]

plt.figure(figsize=(8, 3))
for i in range(8):
    plt.hist(dist_list[i], bins=50, alpha=0.5)
plt.xlabel('Ion-pair distance (nm)')
plt.ylabel('Count')
plt.grid()

The protocol for calculating the free energy profile we use here is exactly the same as the one we used for umbrella sampling in Exercise 2. The only meed to modify the parameters, including the number of samples, and the centers of the umbrella potentials.

In [ ]:
import time
import pymbar
from pymbar import timeseries
import random
import scipy.stats

t0 = time.time()

# Step 1: Setting up
K = 8                                       # number of umbrellas
N_max = 2501                                # number of data points in each timeseries of ion-pair distance
kT = 1.381e-23 * 6.022e23 / 1000 * 300      # 1 kT converted to kJ/mol at 300 K
beta_k = np.ones(K) / kT                    # inverse temperature of simulations (in 1/(kJ/mol)) 
d_min, d_max = 0.25, 0.65                   # minimum and maximum of the CV for plotting the FES
nbins = 50                                  # number of bins for FES
K_k = np.ones(K) * 8000                     # spring constant (in kJ/mol/nm**2) for different simulations
N_k, g_k = np.zeros(K, int), np.zeros(K)    # number of samples and statistical inefficiency of different simulations
d_kn = np.zeros([K, N_max])                 # d_kn[k,n] is the ion-pair distance (in nm) for snapshot n from umbrella simulation k
u_kn = np.zeros([K, N_max])                 # u_kn[k,n] is the reduced potential energy without umbrella restraints of snapshot n of umbrella simulation k
uncorrelated_samples = []                   # Uncorrelated samples of different simulations

# Step 2: Read in and subsample the timeseries
for k in range(K):
    d_kn[k] = np.transpose(np.loadtxt(f'colvar_multi.{k}.dat', comments=['@', '#']))[1]
    N_k[k] = len(d_kn[k])
    d_temp = d_kn[k, 0:N_k[k]]
    g_k[k] = timeseries.statistical_inefficiency(d_temp)     
    print(f"Statistical inefficiency of simulation {k}: {g_k[k]:.3f}")
    indices = timeseries.subsample_correlated_data(d_temp, g=g_k[k]) # indices of the uncorrelated samples
    
    # Update u_kn and d_kn with uncorrelated samples
    N_k[k] = len(indices)    # At this point, N_k contains the number of uncorrelated samples for each state k                
    u_kn[k, 0:N_k[k]] = u_kn[k, indices]
    d_kn[k, 0:N_k[k]] = d_kn[k, indices]
    uncorrelated_samples.append(d_kn[k, indices])

d0_k = np.array([0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6])
N_max = np.max(N_k) # shorten the array size
u_kln = np.zeros([K, K, N_max]) # u_kln[k,l,n] is the reduced potential energy of snapshot n from umbrella simulation k evaluated at umbrella l
u_kn -= u_kn.min()  # shift the minimum of the FES to 0

# Step 3: Bin the data
bin_center_i = np.zeros([nbins])
bin_edges = np.linspace(d_min, d_max, nbins + 1)
for i in range(nbins):
    bin_center_i[i] = 0.5 * (bin_edges[i] + bin_edges[i + 1])
   
# Step 4: Evaluate reduced energies in all umbrellas
for k in range(K):
    for n in range(N_k[k]):
        # Compute minimum-image ion-pair distance deviation from umbrella center l
        dd = d_kn[k,n] - d0_k  # delta d

        # Compute energy of snapshot n from simulation k in umbrella potential l
        u_kln[k,:,n] = u_kn[k,n] + beta_k[k] * (K_k / 2) * dd ** 2

# Step 5: Compute, output, and plot the FES
fes = pymbar.FES(u_kln, N_k, verbose=False)
histo_params = {'bin_edges': bin_edges}
d_n = pymbar.utils.kn_to_n(d_kn, N_k=N_k)
fes.generate_fes(u_kn, d_n, fes_type='histogram', histogram_parameters=histo_params)
results = fes.get_fes(bin_center_i, reference_point="from-lowest", uncertainty_method="analytical")
f_i = results["f_i"]
df_i = results["df_i"]

with open('fes.dat', 'w') as f:
    f.write("# free energy profile (in units of kT), from histogramming\n")
    f.write(f"# {'bin':>8s} {'f':>8s} {'df':>8s} \n")
    for i in range(nbins):
       f.write(f"{bin_center_i[i]:>8.3f} {f_i[i]:>8.3f} {df_i[i]:>8.3f} \n")

plt.figure()
plt.plot(bin_center_i, f_i)
plt.fill_between(bin_center_i, f_i - df_i, f_i + df_i, color='lightgreen')
plt.xlabel('Ion-pair distance (nm)')
plt.ylabel('Free energy (kT)')
plt.grid()

t1 = time.time()
print(f'\nTime elapsed: {t1 - t0:.0f} seconds.')

In [ ]:
fes_MetaD = np.transpose(np.loadtxt('../Exercise_3/mpi_based/fes.dat', comments=['@', '#']))
fes_US = np.transpose(np.loadtxt('../Exercise_2/fes.dat', comments=['@', '#']))
fes_REUS = np.transpose(np.loadtxt('fes.dat', comments=['@', '#']))

fes_US[1] -= min(fes_US[1])
fes_REUS[1] -= min(fes_REUS[1])

kT = 300 * 1.380649E-23 * 6.02214076E23 / 1000   # 1 kT in kJ/mol
plt.plot(fes_MetaD[0], fes_MetaD[1] / kT, label='Metadynamics')
plt.plot(fes_US[0], fes_US[1] - min(fes_US[1]), label='Umbrella sampling')
plt.plot(fes_REUS[0], fes_REUS[1] - min(fes_REUS[1]), label='REUS')

plt.fill_between(fes_US[0], fes_US[1] - fes_US[2], fes_US[1] + fes_US[2], color='lightgreen')
plt.fill_between(fes_REUS[0], fes_REUS[1] - fes_REUS[2], fes_REUS[1] + fes_REUS[2], color='lightgreen')
plt.xlabel('Ion-pair distance (nm)')
plt.ylabel('Free energy (kT)')
plt.legend()
plt.grid()

## 4. References
- The paper that proposed Hamiltonian replica exchange: [Sugita, Yuji, Akio Kitao, and Yuko Okamoto. "Multidimensional replica-exchange method for free-energy calculations." The Journal of chemical physics 113.15 (2000): 6042-6051.](https://doi.org/10.1063/1.1308516)

## Takeaways

- Using Hamiltonian replica exchange, we are able to recover a free energy surface for the NaCl system that is consistent with umbrella sampling and multiple walkers metadynamics!